<a href="https://colab.research.google.com/github/agarwalanant/DeepLearning/blob/master/BoltzmannMachines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!wget http://www.superdatascience.com/wp-content/uploads/2017/03/Boltzmann_Machines.zip
!unzip Boltzmann_Machines.zip -d gdrive/My\ \Drive/DeepLearning

--2018-11-01 14:24:51--  http://www.superdatascience.com/wp-content/uploads/2017/03/Boltzmann_Machines.zip
Resolving www.superdatascience.com (www.superdatascience.com)... 104.27.172.21, 104.27.173.21, 2606:4700:30::681b:ac15, ...
Connecting to www.superdatascience.com (www.superdatascience.com)|104.27.172.21|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.superdatascience.com/wp-content/uploads/2017/03/Boltzmann_Machines.zip [following]
--2018-11-01 14:24:51--  https://www.superdatascience.com/wp-content/uploads/2017/03/Boltzmann_Machines.zip
Connecting to www.superdatascience.com (www.superdatascience.com)|104.27.172.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23195800 (22M) [application/zip]
Saving to: ‘Boltzmann_Machines.zip’

Boltzmann_Machines. 100%[===================>]  22.12M  5.42MB/s    in 4.2s    

2018-11-01 14:24:55 (5.31 MB/s) - ‘Boltzmann_Machines.zip’ saved [23195800/23195800]

Archi

In [2]:
!pip3 install torch torchvision
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

    100% |████████████████████████████████| 519.5MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x59902000 @  0x7fbbdc09a2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 61kB 21.5MB/s 
    100% |████████████████████████████████| 2.0MB 4.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
movies = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-1m/movies.dat", sep ="::",header = None,engine = 'python',encoding = 'latin-1')

In [0]:
users = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-1m/users.dat", sep ="::",header = None,engine = 'python',encoding = 'latin-1')

In [0]:
ratings = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-1m/ratings.dat", sep ="::",header = None,engine = 'python',encoding = 'latin-1')

In [0]:
training_set = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-100k/u1.base", delimiter ='\t')

In [12]:
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user

    100% |████████████████████████████████| 20.9MB 1.4MB/s 
    100% |████████████████████████████████| 5.8MB 5.2MB/s 
    100% |████████████████████████████████| 471kB 21.8MB/s 
    100% |████████████████████████████████| 860kB 14.6MB/s 
  Running setup.py bdist_wheel for jupyter-latex-envs ... - \ done
  Stored in directory: /root/.cache/pip/wheels/65/c1/0f/b90b6d7032c79ee40181fdadb72b2ef9a186f02269d6c52f02
Successfully built jupyter-latex-envs
[I 03:41:50 InstallContribNbextensionsApp] jupyter contrib nbextension install --user
[I 03:41:50 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 03:41:50 InstallContribNbextensionsApp] Installing /usr/local/lib/python3.6/dist-packages/jupyter_contrib_nbextensions/nbextensions/navigation-hotkeys -> navigation-hotkeys
[I 03:41:50 InstallContribNbextensionsApp] Making directory: /root/.local/share/jupyter/nbextensions/navigation-hotkeys/
[I 03:41:50 InstallContribNbextensi

In [13]:
!jupyter nbextension enable varInspector/main

Enabling notebook extension varInspector/main...
      - Validating: OK


In [0]:
training_set = np.array(training_set, dtype = 'int   ')